# Agent 2: Cyber Collector

In this multi-agent system, **`Cyber Collector`** serves as the first agent for information collection and organization, with its main task being to provide high-quality, structured information input for subsequent agents.

In [1]:
# load environment variables from the .env file
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(exclude_managed_identity_credential=True), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

## Define knowledge base

In [3]:
# Upload file and create vector store
file = project_client.agents.upload_file_and_poll(
    file_path="./data/cyber_security_framework.md", purpose="assistants")
print(f"Uploaded file, file ID: {file.id}")

vector_store = project_client.agents.create_vector_store_and_poll(
    file_ids=[file.id], name="my_vectorstore")
print(f"Created vector store, vector store ID: {vector_store.id}")

Uploaded file, file ID: assistant-KgYTjEVsdZUHwa9CA9xMAN
Created vector store, vector store ID: vs_Oc6scMAbyIuNSSOaiKZTRghc


## Create Agent

In [4]:
from azure.ai.projects.models import (
    FileSearchTool,
)

# Create file search tool with resources followed by creating agent
file_search = FileSearchTool(vector_store_ids=[vector_store.id])

cyber_collector = project_client.agents.create_agent(
    model=os.environ["CHAT_MODEL"],
    name="cyber_collector",
    description="The cyber collector can collect information about cyber security from its knowledge base.",
    instructions="Hello, you are helpful assistant and can search information from uploaded files.",
    tools=file_search.definitions,
    tool_resources=file_search.resources,
    # Parameters
    temperature=0.7,
    top_p=0.95,
    # Metadata
    metadata={"group": "internet_threat_analysis"},
)

print(f"Created agent, agent ID: {cyber_collector.id}")

Created agent, agent ID: asst_EVTFqoQY6u3RmYC1f0bt6TkE


In [7]:
agent_id = cyber_collector.id

thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id, role="user", content="Retrieve the latest threat information for cybersecurity"
)
print(f"Created message, ID: {message.id}")

# Create and process assistant run in thread with tools
run = project_client.agents.create_and_process_run(
    thread_id=thread.id, assistant_id=agent_id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Created thread, ID: thread_1nKvPjxwSSIAjIlorR67hsV4
Created message, ID: msg_pfhqWaUDP0uSE9ksbeune1ID
Run finished with status: RunStatus.COMPLETED


In [8]:
# Display all messages in the thread
from IPython.display import Markdown, display
import helper

messages = project_client.agents.list_messages(thread_id=thread.id)

display(Markdown(helper.get_conversation_md(messages)))

# Conversation
___
### **User** (2025-04-09 06:38:19 Coordinated Universal Time)
Retrieve the latest threat information for cybersecurity
___
### **Assistant** (2025-04-09 06:38:25 Coordinated Universal Time)
The document contains valuable insights on cybersecurity metrics and performance indicators, which can help organizations understand their current threat landscape. Here are some key points regarding the latest threat information:

1. **Intrusion Attempts**: There is a detailed tracking of unauthorized access attempts across various system entry points, which is essential for identifying potential threats.

2. **Malicious Traffic Proportion**: Analyzing network traffic patterns helps determine the percentage and nature of potentially harmful activity, offering insights into ongoing cyber threats.

3. **Incident Detection Rate**: The document emphasizes the importance of measuring the success rate of security systems in identifying threats, which is critical for understanding the effectiveness of cybersecurity measures.

4. **Mean Time to Detect/Mean Time to Respond (MTTD/MTTR)**: This metric measures the average duration required to identify security incidents and the subsequent response time, which is vital for mitigating the impact of breaches.

5. **Data Breach Volume**: Understanding the quantity and sensitivity of compromised information is crucial for assessing the overall risk and impact of cyber incidents.

These metrics highlight the ongoing need for organizations to monitor and improve their cybersecurity frameworks actively【4:0†source】.
___

In [9]:
# Print citations from the messages
for citation in messages.file_citation_annotations:
    print(citation)

{'type': 'file_citation', 'text': '【4:0†source】', 'start_index': 1356, 'end_index': 1368, 'file_citation': {'file_id': 'assistant-KgYTjEVsdZUHwa9CA9xMAN'}}


In [10]:
# Get messages from the thread
messages = project_client.agents.list_messages(thread_id=thread.id)
message_data = project_client.agents.list_messages(thread_id=thread.id).data
for msg in message_data:
    print(msg.content)

[{'type': 'text', 'text': {'value': 'The document contains valuable insights on cybersecurity metrics and performance indicators, which can help organizations understand their current threat landscape. Here are some key points regarding the latest threat information:\n\n1. **Intrusion Attempts**: There is a detailed tracking of unauthorized access attempts across various system entry points, which is essential for identifying potential threats.\n\n2. **Malicious Traffic Proportion**: Analyzing network traffic patterns helps determine the percentage and nature of potentially harmful activity, offering insights into ongoing cyber threats.\n\n3. **Incident Detection Rate**: The document emphasizes the importance of measuring the success rate of security systems in identifying threats, which is critical for understanding the effectiveness of cybersecurity measures.\n\n4. **Mean Time to Detect/Mean Time to Respond (MTTD/MTTR)**: This metric measures the average duration required to identify

In [11]:
# Delete the file when done
project_client.agents.delete_vector_store(vector_store.id)
print("Deleted vector store")

project_client.agents.delete_file(file_id=file.id)
print("Deleted file")

# Delete the agent when done
project_client.agents.delete_agent(agent_id)
print("Deleted agent")

Deleted vector store
Deleted file
Deleted agent
